# Spark Learning Note - Basic Structured Operations

## DataFrame vs Dataset

- DataFrames are untyped until runtime - when you execute on it
- Datasets are typed during the compiling time - when you declare it
- Usually just work with DataFrames. When need strict compile-time checking, Dataset is prefered.
- Since Python is Dynamic language, it does not support Datasets (at least for Spark2)

## Structured API Execution Steps

- Write DataFrame/Dataset/SQL code
- Spark convert code to logical plan (analyze and **optimize** the logical plan)
- Spark transform logical plan to physical plan (**optimize** the physical plan, how to execute on cluster)
- Spark Execute physical plan on clusters

In [27]:
from pyspark.sql import SparkSession
data_example_path = '/home/jgeng/Documents/Git/SparkLearning/data/2015flight.csv'

### Spark Session
The entry point into all functionality in Spark is the SparkSession class. To create a basic SparkSession, just use SparkSession.builder:

In [22]:
# build a spark session locally
spark = SparkSession.builder.appName('Spark Example').getOrCreate()
spark

### 1. Read File into DataFrame
Spark session can read file of differemt formats.
- Use `.format()` to specify file format
- `option()` provide many configurations for reading the data such as read header
- DataFrame object has a method `printSchema()`

DataType on Read:
- `json` file contains information regarding the type of the data (but not precision). 
- data in `csv` file will be read as string if not specified
- **the** `option('inferSchema', True)` **will enable the schema inference for reading the csv file**

In [28]:
# read from csv file
df = spark.read.format('csv').load(data_example_path)

In [48]:
# when does not specify the header, spark will treate each row as a data record and add header
df.show(3)
df.printSchema() # count is in StringType!

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
+-----------------+-------------------+-----+
only showing top 3 rows

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: string (nullable = true)



In [54]:
# use option to read header
df = spark.read.format('csv').option('header', True).option('inferSchema', True).load(data_example_path)
df.show(3)
df.printSchema()  # now count is in IntegerType!

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
+-----------------+-------------------+-----+
only showing top 3 rows

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: integer (nullable = true)



In [59]:
# very handy method
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: integer (nullable = true)



### 2. Load File with Manual Schema

When read data from file, if not specifying the schema of your data, default schema-on-read will be created for the data. **This could cause precision issue if the data in file is in different precision. In production, it is usually recommended to manually setup the schema.**

`pyspark.sql.types` have all supported data types. 

For a list of all: https://www.oreilly.com/library/view/spark-the-definitive/9781491912201/ch04.html

Spark DataFrame Schema is defiend by StructType and a list of StructField within it.

StructField:
- name: name of the field/column
- type: data type
- nullable: whether null value is allowed
- metadata: way of storing information about this column (will be used in machine learning)

In [61]:
# a more low level representation
print(df.schema)

StructType(List(StructField(DEST_COUNTRY_NAME,StringType,true),StructField(ORIGIN_COUNTRY_NAME,StringType,true),StructField(count,IntegerType,true)))


In [76]:
# specify a schema manually - e.g. what if count is long rather than integer
from pyspark.sql.types import StructType, StructField, StringType, LongType

# field name, data type, nullable
field_1 = StructField('DEST_COUNTRY_NAME', StringType(), True)
field_2 = StructField('ORIGIN_COUNTRY_NAME', StringType(), True)
field_3 = StructField('count', LongType(), False)

manualSchema = StructType([field_1, field_2, field_3])

In [77]:
df = spark.read.format('csv').option('header', True).schema(manualSchema).load(data_example_path)
df.show(3) 

# now count is long! 
# but nullable is true
# this is because CSV format doesn't provide any tools which allow you to specify data constraints 
# so by definition reader cannot assume that input is not null and your data indeed contains nulls.
df.printSchema()  

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
+-----------------+-------------------+-----+
only showing top 3 rows



### 3. Create Column, Row, and DataFrame 

In [93]:
from pyspark.sql.functions import col
# create colum
col('new_col')

Column<b'new_col'>